# Versuch mit CSF (Cloth Simulation Filter)

In [4]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.spatial import KDTree
import os
import json
import time 
import pyvista as pv
from interessant import * # Bei Änderungen Kernel neu starten

In [5]:
run = run24
#run = run14
# filename = interessant['OLA gleiche Höhe wie Gleis']

# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A (in run24 Achszähler): 6; 
# Viele Gleise: 33; Anfang Weiche: 34; OLA gleiche H: 35; Y: 37
key = list(interessant.keys())[38] 
filename = interessant[key]
print(key, filename)

filename = os.path.join(run, filename)
if not os.path.exists(filename):
    raise FileNotFoundError(filename)

Weiche C 4473850_5336225.copc.laz


In [13]:
pipeline = pdal.Pipeline([pdal.Reader(filename)]) | pdal.Filter.csf() 
# pipeline = pdal.Pipeline([pdal.Reader(filename)]) | pdal.Filter.csf() | pdal.Filter.range(limits="Classification![2:2]") # | pdal.Filter.outlier() | pdal.Filter.range(limits="Classification![7:7]") # | pdal.Filter.radialdensity(radius=0.1)
pipeline = pdal.Pipeline([pdal.Reader(filename)]) | pdal.Filter.csf() | pdal.Filter.hag_delaunay()   

pipeline.execute()
points = pipeline.arrays[0]
print("Number of points:", len(points))

Number of points: 1598139


In [15]:
xyz = np.column_stack((points['X'], points['Y'], points['Z']))
pcd = pv.PolyData(xyz)
pcd['Intensity'] = points['Intensity']
pcd['hag'] = points['HeightAboveGround']
pcd['Classification'] = points['Classification']
# pcd['RadialDensity'] = points['RadialDensity']
pcd.plot(scalars="hag")

Widget(value='<iframe src="http://localhost:43423/index.html?ui=P_0x7f1e880c0160_2&reconnect=auto" class="pyvi…

In [8]:
points.dtype 

dtype([('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('Synthetic', 'u1'), ('KeyPoint', 'u1'), ('Withheld', 'u1'), ('Overlap', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8'), ('ScanChannel', 'u1'), ('Red', '<u2'), ('Green', '<u2'), ('Blue', '<u2')])

In [9]:
def prettyprint(jsonstring, indent=2):
    print(json.dumps(json.loads(jsonstring), indent=indent))


In [10]:
prettyprint(pipeline.toJSON())

[
  {
    "filename": "/media/riannek/minimax/gleis/2024-08-13/01/run24/01/4473850_5336225.copc.laz",
    "tag": "readers_copc1",
    "type": "readers.copc"
  },
  {
    "type": "filters.csf",
    "tag": "filters_csf1"
  }
]


In [11]:
print(json.dumps(json.loads(pipeline.toJSON()), indent=2))

[
  {
    "filename": "/media/riannek/minimax/gleis/2024-08-13/01/run24/01/4473850_5336225.copc.laz",
    "tag": "readers_copc1",
    "type": "readers.copc"
  },
  {
    "type": "filters.csf",
    "tag": "filters_csf1"
  }
]
